# Lecture 10: Constant-temperature molecular dynamics

## Physics 7810, Spring 2020

## 10.1 - Overview
 
Because Hamiltonian dynamics conserves energy, the natural thermodynamic ensemble for molecular dynamics (MD) simulations is the $NVE$ ensemble. In many cases it is more useful to conduct simulations in other ensembles ($NVT$ or $NPT$), as these ensembles correspond to the typical experimental conditions, and may be more convenient for calculating certain quantities (e.g., free energies and phase diagrams).

Here we will focus on methods for carrying out MD simulations at constant temperature $T$, in the $NVT$ ensemble. Methods for maintaining constant pressure in MD simulations will be discussed later.

Before discussing specific constant-$T$ MD methods, it's useful to consider what we are really trying to simulate when we carry out an MD simulation (or any statistical mechanical simulation for that matter). Quite generally, we can regard an MD simulation of a system of $N$ particles with periodic boundary conditions as a model of a small *subsystem* within a much larger, macroscopic system, in which the remainder of the macroscopic system acts as a thermal reservoir (heat bath) that maintains the subsystem at a constant *average* temperature $T$. The macroscopic reservoir may also exchange volume and particles with the subsystem to maintain constant pressure $P$ and chemical potential $\mu$, but for now we will only consider exchange of *energy* between the subsystem and the reservoir.

As pointed out by Frenkel and Smit (see Section 6.1), the *instantaneous* kinetic temperature $T_K$ of such a hypothetical subsystem isn't really constant, but fluctuates around an average value related to the average kinetic energy per particle by

$$
k_B T = k_B \langle T_K \rangle = m \langle v_\alpha^2 \rangle,
$$

where $m$ is the particle mass and $v_\alpha$ is the $\alpha$th component of its velocity, and the angle brackets denote a canonical ($NVT$) ensemble average. In the canonical ensemble, the relative variance in the instantaneous kinetic temperature $T_K$ is

$$
\frac{\sigma_{T_K}^2}{\langle T_K \rangle^2}
\equiv \frac{\langle T_K^2 \rangle - \langle T_K \rangle^2}{\langle T_K \rangle^2} = \frac{2}{3N}.
$$

Thus, so-called *isokinetic* MD methods that hold the kinetic energy rigorously constant do not correctly simulate the canonical ensemble, although they may yield the correct canonical distribution for properties that only depend on particle coordinates (not momenta). As such isokinetic schemes offer no particular advantage in terms of simplicity or efficiency relative to alternative methods, they will not be discussed further here.

A bewildering variety of constant-temperature MD algorithms have been proposed, but these methods can be grouped into two general categories, based on how the simulated system is coupled to the heat bath.

*Stochastic* methods couple the system to the heat bath through stochastic impulsive forces that act on randomly selected particles at random intervals. These stochastic interactions with the heat bath can be regarded as Monte Carlo moves that transport the system from one constant-energy shell to another. The method proposed by Andersen (the *Andersen thermostat*) is an example of a stochastic thermostat that we'll discuss shortly. *Langevin dynamics* is another stochastic dynamics method that we'll discuss later in the semester.

*Deterministic* methods regulate the temperature by introducing additional 'thermostat' degrees of freedom in an extended Lagrangian formulation. The most commonly used extended Lagrangian method is the so-called *Nose-Hoover thermostat* (and the related Nose-Hoover chain algorithm), which will be described in more detail below. Integration schemes for such extended Lagrangians can be derived via the same operator-splitting methods that we used to derive the velocity Verlet integrator.

A variety of *ad hoc* thermostats also exist, most notably the *Berendsen thermostat*, a simple method that's useful for 'rough and ready' equilibration of simulated systems, but doesn't rigorously sample the canonical distribution function. 

## 10.2 - The Andersen thermostat
 
The constant-temperature MD scheme proposed by Andersen involves stochastic 'collisions' of randomly selected particles at a constant frequency $\nu$. If successive collisions are uncorrelated, then the distribution of time intervals $t$ between two successive collisions is a Poisson distribution,

$$
P(t;\nu) = \nu \exp(-\nu t),
$$

and $P(t;\nu) dt$ is the probability that the next collision will take place in the time interval $[t,t+dt]$. The strength of the coupling to the heat bath is determined by the frequency of stochastic collisions $\nu$.

A constant-temperature MD simulation then consists of the following steps:
1. Starting with an initial set of positions and momenta $\left\{ {\bf r}^N(0), {\bf p}^N(0) \right\}$, integrate the equations of motion for a time $\Delta t$ (note that $\Delta t$ could correspond to a single integration timestep or some fixed number of integration timesteps).
2. Select $N \nu \Delta t$ particles at random to undergo a collision with the heat bath (the probability of selecting a given particle in a time interval $\Delta t$ is $\nu \Delta t$).
3. If particle $i$ has been selected to undergo a collision, its new velocity is drawn from a Maxwell-Boltzmann distribution corresponding to the desired temperature $T$.

Mixing Hamiltonian dynamics with stochastic collisions turns a reversible MD simulation into a Markov process.

It can be shown (see Frenkel and Smit, Section 6.1) that the canonical ensemble phase space distribution is invariant under repeated applications of the Andersen algorithm (it's the limiting distribution for the Markov process).

This implies that the Andersen algorithm generates a canonical distribution of particle positions and momenta.

This method *does not* generate the correct dynamics, however. In fact, the dynamical properties will depend on the collision frequency $\nu$.

Velocity distribution in a Lennard-Jones fluid ($T^\ast = 2.0$, $\rho^\ast = 0.8442$, $N = 108$). The solid line is the Maxwell-Boltzmann distribution, and the symbols are from MD simulations using the Andersen algorithm with collision rates of $\nu = 0.01$ and $\nu = 0.001$:

<img src="images/Frenkel_Fig_6.1.png" alt="Drawing" style="width: 600px;">

Figure from *Understanding Molecular Simulation: From Algorithms to Applications*, by Daan Frenkel and Berend Smit.

Equation of state for the Lennard-Jones fluid ($T^\ast = 2.0$, $N = 108$), comparing results from MD simuations using the Andersen thermostat (open symbols) with results from MC simulations (closed symbols) and an equation of state from the literature (solid line):

<img src="images/Frenkel_Fig_6.2.png" alt="Drawing" style="width: 600px;">

Figure from *Understanding Molecular Simulation: From Algorithms to Applications*, by Daan Frenkel and Berend Smit.

Mean-squared displacement as a function of time for various values of the collision frequency $\nu$ in the Lennard-Jones fluid ($T^\ast = 2.0$, $\rho^\ast = 0.8442$, $N = 108$):

<img src="images/Frenkel_Fig_6.3.png" alt="Drawing" style="width: 600px;">

Figure from *Understanding Molecular Simulation: From Algorithms to Applications*, by Daan Frenkel and Berend Smit.

## 10.3 - The Nose-Hoover thermostat
 
A commonly used *deterministic* thermostat for MD simulations is the *Nose-Hoover* thermostat. This is an example of and *extended system* method, in which the exchange of energy with the thermal bath is treated by introducing additional reservoir degrees of freedom. 

In this approach, energy flows between the system and the reservoir, with a dynamics controlled by a parameter that plays the role of 'thermal inertia', analogous to controlling the pressure of a system using a piston. Nose devised a clever way of doing this that can be shown to exactly reproduce the canonical distribution function of the particle positions and momenta, $\left\{ {\bf r}^N, {\bf p}^N \right\}$, but that relies on an awkward rescaling of time that is somewhat inconvenient to use in practice. For this reason, a modified version of the algorithm proposed by Hoover that avoids time scaling (the Nose-Hoover algorithm) is used instead.

In the Nose-Hoover method, the equations of motion for particle and reservoir degrees are freedom are derived from an *extended Lagrangian*. The derivation of the method and the proof that this method samples the canonical distribution function are rather involved, and won't be repeated here. For details, refer to Section 6.1.2 in Frenkel and Smit.

The Nose-Hoover equations of motion are:

$$
\dot{\bf r}_i = \frac{{\bf p}_i}{m}
$$
$$
\dot{\bf p}_i = {\bf f}_i - \frac{p_\eta}{Q} {\bf p}_i
$$
$$
\dot{\eta} = \frac{p_\eta}{Q}
$$
$$
\dot{p}_\eta = \sum_{i = 1}^N \frac{{\bf p}_i \cdot {\bf p}_i}{m} - g k_B T.
$$

Here, $g$ is the number of degrees of freedom, e.g., $g = 3N -3$ for a 3D system in which the total momentum is conserved, $\eta$ is the thermostat 'displacement', $p_\eta$ is the corresponding thermostat momentum, and $Q$ is the thermostat 'mass'. The right-hand side of the fourth equation represents a sort of thermal 'force', as it measures how much the total kinetic energy deviates from the average value expected from equipartition. From the second equation, we can see that the combination $p_\eta / Q = \dot{\eta} = \xi$ plays the role of a friction coefficient, which increases if the system is too hot and decreases (and may go negative) if the system is too cold.

These equations of motion are non-Hamiltonian, in that they cannot be derived from a Hamiltonian. Nevertheless, the equations of motion do conserve an energy-like quantity,

$$
H_\mathrm{NH} = K({\bf p}^N) + U({\bf r}^N) + \frac{p_\eta^2}{2 Q} + g k_B T \eta.
$$

Verifying that this quantity is conserved is a useful check on the dynamics, to make sure that the algorithm has been  implemented correctly, and that the integration timestep isn't too large.

Numerical integration schemes for the Nose-Hoover equations of motion can be derived using an operator-splitting approach similar to that used to derive the velocity Verlet algorithm. Although the equations of motion are non-Hamiltonian, it's possible to define an appropriate phase-space volume that is conserved by the dynamics, yielding a integration scheme that is stable over long timescales.

As for a classical Hamiltonian system, we can define a classical time-evolution operator $\exp(iLt)$ for the extended system,

$$
\Gamma(t) = e^{iLt} \Gamma(0),
$$

where $\Gamma$ denotes the extended set of phase space variables, $\Gamma(t) = ({\bf p}^N(t), {\bf q}^N(t),\eta(t),p_\eta(t))$, and the Liouville operator is defined as

$$
iL = \dot{\Gamma} \frac{\partial}{\partial \Gamma}.
$$

To derive an integrator, we use the splitting $iL = iL_1 + iL_2 + iL_3 + iL_4$, where

$$
iL_1 = \frac{{\bf p}}{m} \cdot \frac{\partial}{\partial {\bf r}},
$$

$$
iL_2 = {\bf f} \cdot \frac{\partial}{\partial {\bf p}},
$$

$$
iL_3 = \xi \frac{\partial}{\partial \eta} - \xi {\bf p} \cdot \frac{\partial}{\partial {\bf p}},
$$

$$
iL_4 = \left( \frac{{\bf p} \cdot {\bf p}}{m} - g k_B T \right) \frac{\partial}{\partial p_\eta},
$$

where $\xi = p_\eta / Q$, and we've suppressed the particle index $i$. Note that the two differential operators in $iL_3$ commute.

With this splitting, the action of the corresponding propagators is easily obtained, 

$$
e^{iL_1 t} {\bf r} = \exp \left(t \frac{{\bf p}}{m} \cdot \frac{\partial}{\partial {\bf r}} \right) {\bf r}
= {\bf r} + \frac{{\bf p}}{m} t,
$$

$$
e^{iL_2 t} {\bf p} = \exp \left(t {\bf f} \cdot \frac{\partial}{\partial {\bf p}} \right) {\bf p}
= {\bf p} + {\bf f} t,
$$

$$
e^{iL_3 t} \left( \begin{array}{c} \eta \\ {\bf p} \end{array} \right)
= \exp \left(t \xi \frac{\partial}{\partial \eta} - t \xi {\bf p} \cdot \frac{\partial}{\partial {\bf p}} \right)
\left( \begin{array}{c} \eta \\ {\bf p} \end{array} \right)
= \left( \begin{array}{c} \eta + \xi t \\ {\bf p} e^{-\xi t} \end{array} \right),
$$

$$
e^{iL_4 t} p_\eta
= \exp \left[t \left( \frac{{\bf p} \cdot {\bf p}}{m} - g k_B T \right) \frac{\partial}{\partial p_\eta} \right] p_\eta
= p_\eta + \left( \frac{{\bf p} \cdot {\bf p}}{m} - g k_B T \right) t.
$$

Here we only show the action on those variables that are changed, and use the identity

$$
\exp \left( a x \frac{\partial}{\partial x} \right) f(x)
= \exp \left( a \frac{\partial}{\partial \ln x} \right) f[ \exp ( \ln x)]
= f[\exp ( \ln x + a)] = f[x \exp (a)]
$$

We note in passing that the previous identity is a special case of a more general result,

$$
\exp \left( a \frac{\partial}{\partial g(x)} \right) f(x)
= \exp \left( a \frac{\partial}{\partial g(x)} \right) f \left( g^{-1} \left[ g(x) \right] \right)
$$

$$
= \exp \left( a \frac{\partial}{\partial y} \right) f \left( g^{-1} \left[ y \right] \right)
= f \left( g^{-1} \left[ y + a \right] \right)
= f \left( g^{-1} \left[ g(x) + a \right] \right).
$$

This more general result will come in handy later on.

There are many possible ways of splitting the total time-evolution operator to obtain a discrete-time approximation. One of the most convenient options is that of Martyna et al. (*Molec. Phys.* **87**, 1117 (1996)),

$$
e^{iL \Delta t} = \left[ e^{iL_4 \frac{\Delta t}{4}} e^{iL_3 \frac{\Delta t}{2}} e^{iL_4 \frac{\Delta t}{4}} \right]
\left[ e^{iL_2 \frac{\Delta t}{2}} e^{iL_1 \Delta t} e^{iL_2 \frac{\Delta t}{2}} \right]
\left[ e^{iL_4 \frac{\Delta t}{4}} e^{iL_3 \frac{\Delta t}{2}} e^{iL_4 \frac{\Delta t}{4}} \right].
$$

The discrete-time propagator in the middle set of square brackets is just the velocity Verlet integrator, so this algorithm can be implemented in a straightforward way, by bracketing the velocity Verlet integrator between propagators involving the reservoir variables. 

Looking back at my most recent implementation of the Nose-Hoover thermostat, I see that I used a slightly different operator splitting,

$$
e^{iL \Delta t} = \left[ e^{iL_4 \frac{\Delta t}{2}} e^{iL_3 \frac{\Delta t}{2}} \right]
\left[ e^{iL_2 \frac{\Delta t}{2}} e^{iL_1 \Delta t} e^{iL_2 \frac{\Delta t}{2}} \right]
\left[ e^{iL_3 \frac{\Delta t}{2}} e^{iL_4 \frac{\Delta t}{2}} \right].
$$

This is a bit simpler, but the integrator for the thermostat degrees of freedom is of lower order in $\Delta t$ in this formulation. Either operator splitting scheme is very easily implemented in code, so the higher-accuracy version may be slightly preferred.

Velocity distribution in a 3D Lennard-Jones fluid ($T^\ast = 1.0$, $\rho^\ast = 0.75$, $N = 256$) from MD simulations using the Nose-Hoover thermostat for two different values of thermostat 'mass' ($Q = 0.1$ and $Q = 1.0$), compared with the Maxwell-Boltzmann distribution:

<img src="images/Frenkel_Fig_6.4.png" alt="Drawing" style="width: 600px;">

Figure from *Understanding Molecular Simulation: From Algorithms to Applications*, by Daan Frenkel and Berend Smit.

Response of the Lennard-Jones fluid ($\rho^\ast = 0.75$, $N = 256$) to a sudden increase in the imposed temperature as a function of time for various values of the Nose-Hoover coupling constant ('mass') $Q$:

<img src="images/Frenkel_Fig_6.5.png" alt="Drawing" style="width: 600px;">

Figure from *Understanding Molecular Simulation: From Algorithms to Applications*, by Daan Frenkel and Berend Smit.

Effect of coupling constant $Q$ on the mean-squared displacement (MSD) of the Lennard-Jones fluid ($T^\ast = 1.0$, $\rho^\ast = 0.75$, $N = 256$):

<img src="images/Frenkel_Fig_6.6.png" alt="Drawing" style="width: 600px;">

Note the the MSD is relatively insensitive to $Q$, in contrast to the Andersen thermostat, where the MSD has a sensitive dependence on the velocity resampling frequency $\nu$.

Figure from *Understanding Molecular Simulation: From Algorithms to Applications*, by Daan Frenkel and Berend Smit.

Simulated phase space trajectories for the 1D harmonic oscillator in the microcanonical ensemble (left), using the Andersen thermostat (center), and using the Nose-Hoover method (right), where the horizontal axis is the position and the vertical axis is the velocity:

<img src="images/Frenkel_Fig_6.7.png" alt="Drawing" style="width: 1200px;">

The Andersen thermostat yields the correct canonical (Gaussian) phase distribution, but the Nose-Hoover thermostat does not. Such *non-ergodic behavior*, which can occur in small or stiff systems, can be ameliorated by coupling the thermostat to another thermostat or, if necessary to a chain of thermostats, the so-called *Nose-Hoover chain* method.

Figure from *Understanding Molecular Simulation: From Algorithms to Applications*, by Daan Frenkel and Berend Smit.

## 10.4 - The Berendsen thermostat
 
Another deterministic method for controlling the temperature is the Berendsen thermostat ('weak coupling' method). In this method, momenta are scaled at each timestep,

$$
{\bf p}^\prime = {\bf p} \sqrt{1 + \frac{\Delta t}{\tau} \left( \frac{T}{T_\mathrm{kin}} - 1 \right)},
$$

where $\tau$ is a judiciously chosen time constant, and $T_\mathrm{kin}$ is the instantaneous kinetic temperature. Note that the momenta are scaled down if the $T_\mathrm{kin}$ exceeds the target temperature $T$, and are scaled up if $T_\mathrm{kin} < T$, and the temperature will in principle relax to (and fluctuate around) the target temperature.

The Berendsen thermostat is easy to implement, but yields equations of motion that aren't time-reversible, and yields a phase space distribution that isn't canonical. However, it can be useful for 'rough and ready' equilibration in MD simulations. Full disclosure: I've used this thermostat and the corresponding weak coupling method for controlling pressure (the *Berendsen barostat*, to be discussed later on) extensively in the past.